<a href="https://colab.research.google.com/github/ashimasinghal/Indian-NER/blob/main/Hindi_NER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install seqeval
!pip install transformers

     |████████████████████████████████| 43 kB 923 kB/s 
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16181 sha256=bda62e8177f9adeff0121da1fa399bb38ac1ccb29c074a4408b6590fa899f859
  Stored in directory: /root/.cache/pip/wheels/05/96/ee/7cac4e74f3b19e3158dce26a20a1c86b3533c43ec72a549fd7
Successfully built seqeval
     |████████████████████████████████| 3.1 MB 4.0 MB/s 
     |████████████████████████████████| 3.3 MB 47.6 MB/s 
     |████████████████████████████████| 895 kB 37.2 MB/s 
     |████████████████████████████████| 59 kB 6.7 MB/s 
     |████████████████████████████████| 596 kB 50.4 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:

import pandas as pd
import math
import numpy as np
from seqeval.metrics import f1_score
from seqeval.metrics import classification_report,accuracy_score,f1_score
import torch.nn.functional as F

In [ ]:
import torch
import os
from tqdm import tqdm,trange
from torch.optim import Adam
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertConfig
from transformers import BertForTokenClassification, AdamW

In [ ]:
!pip list | grep -E 'transformers|torch|Keras'

Keras-Preprocessing           1.1.2
torch                         1.10.0+cu111
torchsummary                  1.5.1
torchtext                     0.11.0
torchvision                   0.11.1+cu111
transformers                  4.12.3


In [ ]:
# load data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import numpy as np
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/BERT/NER Corpus/Hindi.csv')

Mounted at /content/drive


In [ ]:
df

,Unnamed: 0,word,T1,T2,T3,T4,T5
0,0,],SYM,I-NP,o,o,o
1,1,',SYM,I-NP,o,o,o
2,2,',SYM,I-NP,o,o,o
3,3,',SYM,I-NP,o,o,o
4,4,चलभाष,NNP,I-NP,o,o,o
...,...,...,...,...,...,...,...
113148,113148,"1,585",QC,B-BLK,B-COUNT,o,o
113149,113149,.,SYM,I-BLK,o,o,o
113150,113150,.,SYM,B-BLK,o,o,o
113151,113151,"1,579",NNP,B-NP,B-COUNT,o,o


In [ ]:
sentence_id = []
words = df['word']
sentence = 1
for word in words:
  sentence_id.append(sentence)
  if(word=='.'):
    sentence += 1

In [ ]:
df.insert(1,'Sentence ID',sentence_id,allow_duplicates=True)
df.head(50)

,Unnamed: 0,Sentence ID,word,T1,T2,T3,T4,T5
0,0,1,],SYM,I-NP,o,o,o
1,1,1,',SYM,I-NP,o,o,o
2,2,1,',SYM,I-NP,o,o,o
3,3,1,',SYM,I-NP,o,o,o
4,4,1,चलभाष,NNP,I-NP,o,o,o
5,5,1,',SYM,I-NP,o,o,o
6,6,1,',SYM,I-NP,o,o,o
7,7,1,',SYM,I-NP,o,o,o
8,8,1,(,NNPC,B-NP,o,o,o
9,9,1,मोबाइल,NNC,I-NP,o,o,o


In [ ]:
hindi = pd.DataFrame()
hindi = df[['Sentence ID','word','T2','T3']]
hindi.rename(columns={'Sentence ID': 'Sentence', 'T2': 'POS','T3':'TAG'}, inplace=True)
hindi

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,Sentence,word,POS,TAG
0,1,],I-NP,o
1,1,',I-NP,o
2,1,',I-NP,o
3,1,',I-NP,o
4,1,चलभाष,I-NP,o
...,...,...,...,...
113148,9472,"1,585",B-BLK,B-COUNT
113149,9472,.,I-BLK,o
113150,9473,.,B-BLK,o
113151,9474,"1,579",B-NP,B-COUNT


In [ ]:
hindi.POS.unique()

array(['I-NP', 'B-NP', 'B-BLK', 'B-VGF', 'I-VGF', 'B-NEGP', 'I-NEGP',
       'I-BLK', 'B-JJP', 'I-JJP', 'B-CCP', 'B-RBP', 'B-FRAGP', 'I-FRAGP',
       'B-VGNN', 'I-VGNN', 'B-VGNF', 'I-VGNF', 'I-RBP', 'I-CCP', 'BLK',
       'NP'], dtype=object)

In [ ]:
hindi.TAG.unique()

array(['o', 'B-ARTIFACT', 'I-ARTIFACT', 'B-PERSON', 'I-PERSON', 'B-DATE',
       'B-MONTH', 'B-YEAR', 'B-LOCATION', 'B-COUNT', 'B-ORGANIZATION',
       'I-ORGANIZATION', 'B-MATERIALS', 'I-MATERIALS', 'I-COUNT',
       'B-MONEY', 'I-MONEY', 'B-ENTERTAINMENT', 'I-ENTERTAINMENT',
       'I-LOCATION', 'B-PERIOD', 'I-PERIOD', 'B-LIVTHINGS', 'I-YEAR',
       'B-DISTANCE', 'I-DISTANCE', 'B-PLANTS', 'B-SDAY', 'I-SDAY',
       'B-DAY', 'B-FACILITIES', 'I-FACILITIES', 'I-DATE', 'B-DISEASE',
       'I-DISEASE', 'I-MONTH', 'I-LIVTHINGS', 'B-TIME', 'I-TIME',
       'B-QUANTITY', 'I-QUANTITY', 'B-LOCOMOTIVE', 'I-LOCOMOTIVE',
       'I-DAY', 'I-PLANTS'], dtype=object)

In [ ]:
hindi['Sentence'].nunique(), hindi.word.nunique(), hindi.POS.nunique(), hindi.TAG.nunique()

(9474, 15735, 22, 45)

In [ ]:

hindi.TAG.value_counts()

o                  100357
B-LOCATION           2160
B-PERSON             2057
B-COUNT              1747
B-YEAR                709
I-PERSON              667
B-ARTIFACT            624
I-LOCATION            548
B-ENTERTAINMENT       332
B-DISEASE             328
I-ARTIFACT            280
I-ENTERTAINMENT       273
I-PERIOD              271
I-COUNT               247
I-ORGANIZATION        235
B-MATERIALS           208
B-PERIOD              199
B-LIVTHINGS           190
B-ORGANIZATION        169
I-DISEASE             161
I-DISTANCE            155
B-DISTANCE            127
I-FACILITIES          122
I-TIME                114
B-MONTH               106
B-SDAY                102
B-FACILITIES           95
B-TIME                 88
I-MATERIALS            60
B-QUANTITY             53
I-QUANTITY             51
I-LIVTHINGS            42
B-PLANTS               35
B-LOCOMOTIVE           34
B-MONEY                28
I-YEAR                 28
I-MONEY                27
B-DATE                 26
B-DAY       

In [ ]:
# parsing the data


In [ ]:
# import numpy as np
# from sklearn.model_selection import train_test_split
# hindi_train, hindi_test = train_test_split( hindi, test_size=0.33, random_state=42)

In [ ]:
class SentenceGetter(object):
    
    def __init__(self, data):
        self.n_sent = 1
        self.data = data
        self.empty = False
        agg_func = lambda s: [(w, p, t) for w, p, t in zip(s["word"].values.tolist(),
                                                           s["POS"].values.tolist(),
                                                           s["TAG"].values.tolist())]
        self.grouped = self.data.groupby("Sentence").apply(agg_func)
        self.sentences = [s for s in self.grouped]
    
    def get_next(self):
        try:
            s = self.grouped["Sentence: {}".format(self.n_sent)]
            self.n_sent += 1
            return s
        except:
            return None

In [ ]:
getter = SentenceGetter(hindi)

In [ ]:
sentences = [[s[0] for s in sent] for sent in getter.sentences]
sentences[0]

[']',
 "'",
 "'",
 "'",
 'चलभाष',
 "'",
 "'",
 "'",
 '(',
 'मोबाइल',
 'भाष',
 ',',
 'मोबाइल',
 ',',
 'सेलभाष',
 ',',
 '.']

In [ ]:
poses = [[s[1] for s in sent] for sent in getter.sentences]
print(poses[0])

['I-NP', 'I-NP', 'I-NP', 'I-NP', 'I-NP', 'I-NP', 'I-NP', 'I-NP', 'B-NP', 'I-NP', 'I-NP', 'I-NP', 'B-NP', 'I-NP', 'B-NP', 'I-NP', 'I-NP']


In [ ]:
labels = [[s[2] for s in sent] for sent in getter.sentences]
print(labels[0])

['o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o']


In [ ]:
# Make TAG name into index for training
tags_vals = list(set(hindi["TAG"].values))
tags_vals.append('X')
tags_vals.append('[CLS]')
tags_vals.append('[SEP]')
tags_vals = set(tags_vals)
tags_vals

{'B-ARTIFACT',
 'B-COUNT',
 'B-DATE',
 'B-DAY',
 'B-DISEASE',
 'B-DISTANCE',
 'B-ENTERTAINMENT',
 'B-FACILITIES',
 'B-LIVTHINGS',
 'B-LOCATION',
 'B-LOCOMOTIVE',
 'B-MATERIALS',
 'B-MONEY',
 'B-MONTH',
 'B-ORGANIZATION',
 'B-PERIOD',
 'B-PERSON',
 'B-PLANTS',
 'B-QUANTITY',
 'B-SDAY',
 'B-TIME',
 'B-YEAR',
 'I-ARTIFACT',
 'I-COUNT',
 'I-DATE',
 'I-DAY',
 'I-DISEASE',
 'I-DISTANCE',
 'I-ENTERTAINMENT',
 'I-FACILITIES',
 'I-LIVTHINGS',
 'I-LOCATION',
 'I-LOCOMOTIVE',
 'I-MATERIALS',
 'I-MONEY',
 'I-MONTH',
 'I-ORGANIZATION',
 'I-PERIOD',
 'I-PERSON',
 'I-PLANTS',
 'I-QUANTITY',
 'I-SDAY',
 'I-TIME',
 'I-YEAR',
 'X',
 '[CLS]',
 '[SEP]',
 'o'}

In [ ]:
tag2idx={'B-ARTIFACT': 45,
 'B-COUNT': 22,
 'B-DATE': 7,
 'B-DAY': 6,
 'B-DISEASE': 28,
 'B-DISTANCE': 19,
 'B-ENTERTAINMENT': 24,
 'B-FACILITIES': 30,
 'B-LIVTHINGS': 31,
 'B-LOCATION': 0,
 'B-LOCOMOTIVE': 34,
 'B-MATERIALS': 36,
 'B-MONEY': 26,
 'B-MONTH' : 5,
 'B-ORGANIZATION':2,
 'B-PERIOD': 14,
 'B-PERSON': 15,
 'B-PLANTS': 38,
 'B-QUANTITY': 17,
 'B-SDAY': 40,
 'B-TIME':4,
 'B-YEAR': 21,
 'I-ARTIFACT': 46,
 'I-COUNT': 23,
 'I-DATE': 11,
 'I-DAY': 10,
 'I-DISEASE': 29,
 'I-DISTANCE': 20,
 'I-ENTERTAINMENT': 25,
 'I-FACILITIES': 32,
 'I-LIVTHINGS': 33,
 'I-LOCATION': 12,
 'I-LOCOMOTIVE': 35,
 'I-MATERIALS': 37,
 'I-MONEY': 27,
 'I-MONTH': 8,
 'I-ORGANIZATION':3,
 'I-PERIOD': 13,
 'I-PERSON': 16,
 'I-PLANTS': 39,
 'I-QUANTITY': 18,
 'I-SDAY': 40,
 'I-TIME': 1,
 'I-YEAR': 9,
 'X': 41,
 'o': 42,
 '[CLS]': 43,
 '[SEP]':44,
 }
tag2idx

{'B-ARTIFACT': 45,
 'B-COUNT': 22,
 'B-DATE': 7,
 'B-DAY': 6,
 'B-DISEASE': 28,
 'B-DISTANCE': 19,
 'B-ENTERTAINMENT': 24,
 'B-FACILITIES': 30,
 'B-LIVTHINGS': 31,
 'B-LOCATION': 0,
 'B-LOCOMOTIVE': 34,
 'B-MATERIALS': 36,
 'B-MONEY': 26,
 'B-MONTH': 5,
 'B-ORGANIZATION': 2,
 'B-PERIOD': 14,
 'B-PERSON': 15,
 'B-PLANTS': 38,
 'B-QUANTITY': 17,
 'B-SDAY': 40,
 'B-TIME': 4,
 'B-YEAR': 21,
 'I-ARTIFACT': 46,
 'I-COUNT': 23,
 'I-DATE': 11,
 'I-DAY': 10,
 'I-DISEASE': 29,
 'I-DISTANCE': 20,
 'I-ENTERTAINMENT': 25,
 'I-FACILITIES': 32,
 'I-LIVTHINGS': 33,
 'I-LOCATION': 12,
 'I-LOCOMOTIVE': 35,
 'I-MATERIALS': 37,
 'I-MONEY': 27,
 'I-MONTH': 8,
 'I-ORGANIZATION': 3,
 'I-PERIOD': 13,
 'I-PERSON': 16,
 'I-PLANTS': 39,
 'I-QUANTITY': 18,
 'I-SDAY': 40,
 'I-TIME': 1,
 'I-YEAR': 9,
 'X': 41,
 '[CLS]': 43,
 '[SEP]': 44,
 'o': 42}

In [ ]:
tag2name={tag2idx[key] : key for key in tag2idx.keys()}

In [ ]:
# training

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
n_gpu

1

In [ ]:
# vocabulary = "/content/drive/MyDrive/BERT/NER Corpus/bert-base-multilingual-cased/bert-base-multilingual-cased-vocab.txt"

In [ ]:
from transformers import AutoTokenizer, AutoModelForMaskedLM
max_len = 45 
tokenizer = AutoTokenizer.from_pretrained("skillzzzzzy/urberto")

Downloading:   0%|          | 0.00/671 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/795k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/596k [00:00<?, ?B/s]

In [ ]:
# max_len  = 45
# tokenizer=BertTokenizer.from_pretrained('bert-base-multilingual-cased', truncation=True, do_lower_case=True)
# tokenizer = AutoTokenizer.from_pretrained("Davlan/bert-base-multilingual-cased-ner-hrl")
# model = AutoModelForTokenClassification.from_pretrained("Davlan/bert-base-multilingual-cased-ner-hrl")

Downloading:   0%|          | 0.00/972k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.87M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

In [ ]:
tokenized_texts = []
word_piece_labels = []
i_inc = 0
for word_list,label in (zip(sentences,labels)):
    temp_lable = []
    temp_token = []
    
    # Add [CLS] at the front 
    temp_lable.append('[CLS]')
    temp_token.append('[CLS]')
    
    for word,lab in zip(word_list,label):
        token_list = tokenizer.tokenize(word)
        for m,token in enumerate(token_list):
            temp_token.append(token)
            if m==0:
                temp_lable.append(lab)
            else:
                temp_lable.append('X')  
                
    # Add [SEP] at the end
    temp_lable.append('[SEP]')
    temp_token.append('[SEP]')
    
    tokenized_texts.append(temp_token)
    word_piece_labels.append(temp_lable)
    
    if 5 > i_inc:
        print("No.%d,len:%d"%(i_inc,len(temp_token)))
        print("texts:%s"%(" ".join(temp_token)))
        print("No.%d,len:%d"%(i_inc,len(temp_lable)))
        print("lables:%s"%(" ".join(temp_lable)))
    i_inc +=1

No.0,len:60
texts:[CLS] ] ' ' ' à¤ ļ à¤ ² à¤ Ń à¤¾ à¤ · ' ' ' ( à¤ ® à¥ ĭ à¤ ¬ à¤¾ à¤ ĩ à¤ ² à¤ Ń à¤¾ à¤ · , à¤ ® à¥ ĭ à¤ ¬ à¤¾ à¤ ĩ à¤ ² , à¤ ¸ à¥ĩ à¤ ² à¤ Ń à¤¾ à¤ · , . [SEP]
No.0,len:60
lables:[CLS] o o o o o X X X X X X X X o o o o o X X X X X X X X X X o X X X X o o X X X X X X X X X X o o X X X X X X X X X o o [SEP]
No.1,len:3
texts:[CLS] . [SEP]
No.1,len:3
lables:[CLS] o [SEP]
No.2,len:3
texts:[CLS] . [SEP]
No.2,len:3
lables:[CLS] o [SEP]
No.3,len:3
texts:[CLS] . [SEP]
No.3,len:3
lables:[CLS] o [SEP]
No.4,len:3
texts:[CLS] . [SEP]
No.4,len:3
lables:[CLS] o [SEP]


In [ ]:
# Make text token into id
input_ids = pad_sequences([tokenizer.convert_tokens_to_ids(txt) for txt in tokenized_texts],
                          maxlen=max_len, dtype="long", truncating="post", padding="post")
print(input_ids[0])

[    3    65    11    11    11  6843   253  6843   115  6843   260 21507
  6843   120    11    11    11    12  6843   111  9916   238  6843   110
 21507  6843   234  6843   115  6843   260 21507  6843   120    16  6843
   111  9916   238  6843   110 21507  6843   234  6843]


In [ ]:
# Make label into id, pad with "O" meaning others
tags = pad_sequences([[tag2idx.get(l) for l in lab] for lab in word_piece_labels],
                     maxlen=max_len, value=tag2idx["o"], padding="post",
                     dtype="long", truncating="post")
print(tags[0])


[43 42 42 42 42 42 41 41 41 41 41 41 41 41 42 42 42 42 42 41 41 41 41 41
 41 41 41 41 41 42 41 41 41 41 42 42 41 41 41 41 41 41 41 41 41]


In [ ]:
# For fine tune of predict, with token mask is 1,pad token is 0
attention_masks = [[int(i>0) for i in ii] for ii in input_ids]
attention_masks[0];

In [ ]:
# Since only one sentence, all the segment set to 0
segment_ids = [[0] * len(input_id) for input_id in input_ids]
segment_ids[0];

In [ ]:
tr_inputs, val_inputs, tr_tags, val_tags,tr_masks, val_masks,tr_segs, val_segs = train_test_split(input_ids, tags,attention_masks,segment_ids, 
                                                            random_state=4, test_size=0.3)

In [ ]:
len(tr_inputs),len(val_inputs),len(tr_segs),len(val_segs)

(6631, 2843, 6631, 2843)

In [ ]:
tr_inputs = torch.tensor(tr_inputs)
val_inputs = torch.tensor(val_inputs)
tr_tags = torch.tensor(tr_tags)
val_tags = torch.tensor(val_tags)
tr_masks = torch.tensor(tr_masks)
val_masks = torch.tensor(val_masks)
tr_segs = torch.tensor(tr_segs)
val_segs = torch.tensor(val_segs)

In [ ]:
batch_num = 32

In [ ]:
train_data = TensorDataset(tr_inputs, tr_masks, tr_tags)
train_sampler = RandomSampler(train_data)
# Drop last can make batch training better for the last one
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_num,drop_last=True)

valid_data = TensorDataset(val_inputs, val_masks, val_tags)
valid_sampler = SequentialSampler(valid_data)
valid_dataloader = DataLoader(valid_data, sampler=valid_sampler, batch_size=batch_num)

In [ ]:
# model_file_address = '/content/drive/MyDrive/BERT/NER Corpus/bert-base-multilingual-cased'
model = AutoModelForMaskedLM.from_pretrained("skillzzzzzy/urberto", num_labels=len(tag2idx))
# model = BertForTokenClassification.from_pretrained("bert-base-multilingual-cased",num_labels=len(tag2idx))

Downloading:   0%|          | 0.00/256M [00:00<?, ?B/s]

In [ ]:
m_labels = len(tag2idx)

In [ ]:
model;

In [ ]:
model.cuda();

In [ ]:
epochs = 20
max_grad_norm = 1.0

In [ ]:
if n_gpu >1:
    model = torch.nn.DataParallel(model)

In [ ]:
num_train_optimization_steps = int( math.ceil(len(tr_inputs) / batch_num) / 1) * epochs

In [ ]:
# fine tuning
FULL_FINETUNING = True

In [ ]:
if FULL_FINETUNING:
    # Fine tune model all layer parameters
    param_optimizer = list(model.named_parameters())
    no_decay = ['bias', 'gamma', 'beta']
    optimizer_grouped_parameters = [
        {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
         'weight_decay_rate': 0.01},
        {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
         'weight_decay_rate': 0.0}
    ]
else:
    # Only fine tune classifier parameters
    param_optimizer = list(model.classifier.named_parameters()) 
    optimizer_grouped_parameters = [{"params": [p for n, p in param_optimizer]}]
optimizer = AdamW(optimizer_grouped_parameters, lr=3e-5)

In [ ]:
model.train();

In [ ]:
print("***** Running training *****")
print("  Num examples = %d"%(len(tr_inputs)))
print("  Batch size = %d"%(batch_num))
print("  Num steps = %d"%(num_train_optimization_steps))
for _ in trange(epochs,desc="Epoch"):
    tr_loss = 0
    nb_tr_examples, nb_tr_steps = 0, 0
    for step, batch in enumerate(train_dataloader):
        # add batch to gpu
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch
        
        # forward pass
        outputs = model(b_input_ids, token_type_ids=None,
        attention_mask=b_input_mask, labels=b_labels)
        loss, scores = outputs[:2]
        if n_gpu>1:
            # When multi gpu, average it
            loss = loss.mean()
        
        # backward pass
        loss.backward()
        
        # track train loss
        tr_loss += loss.item()
        nb_tr_examples += b_input_ids.size(0)
        nb_tr_steps += 1
        
        # gradient clipping
        torch.nn.utils.clip_grad_norm_(parameters=model.parameters(), max_norm=max_grad_norm)
        
        # update parameters
        optimizer.step()
        optimizer.zero_grad()
        
    # print train loss per epoch
    print("Train loss: {}".format(tr_loss/nb_tr_steps))

***** Running training *****
  Num examples = 6631
  Batch size = 32
  Num steps = 4160


Epoch:   5%|▌         | 1/20 [01:24<26:46, 84.53s/it]

Train loss: 0.569116329225365


Epoch:  10%|█         | 2/20 [02:48<25:19, 84.40s/it]

Train loss: 0.16525093350865414


...

Train loss: 0.031232782464096512


Epoch:  95%|█████████▌| 19/20 [26:40<01:24, 84.25s/it]

Train loss: 0.02815081368555913


Epoch: 100%|██████████| 20/20 [28:05<00:00, 84.25s/it]

Train loss: 0.025632590289421126


In [ ]:
bert_out_address = 'ashiNLP/bert_hindberto_model/hindi'

In [ ]:
if not os.path.exists(bert_out_address):
        os.makedirs(bert_out_address)

In [ ]:
model_to_save = model.module if hasattr(model, 'module') else model

In [ ]:
output_model_file = os.path.join(bert_out_address, "pytorch_model.bin")
output_config_file = os.path.join(bert_out_address, "config.json")

In [ ]:
torch.save(model_to_save.state_dict(), output_model_file)
model_to_save.config.to_json_file(output_config_file)
tokenizer.save_vocabulary(bert_out_address)

('ashiNLP/bert_hindberto_model/hindi/vocab.json',
 'ashiNLP/bert_hindberto_model/hindi/merges.txt')

In [ ]:
# load model
model = AutoModelForMaskedLM.from_pretrained(bert_out_address,num_labels=len(tag2idx))

In [ ]:
model.cuda();

In [ ]:
if n_gpu >1:
    model = torch.nn.DataParallel(model)

In [ ]:
# EVALUATION

In [ ]:
model.eval();

In [ ]:

eval_loss, eval_accuracy = 0, 0
nb_eval_steps, nb_eval_examples = 0, 0
y_true = []
y_pred = []

print("***** Running evaluation *****")
print("  Num examples ={}".format(len(val_inputs)))
print("  Batch size = {}".format(batch_num))
for step, batch in enumerate(valid_dataloader):
    batch = tuple(t.to(device) for t in batch)
    input_ids, input_mask, label_ids = batch
    
#     if step > 2:
#         break
    
    with torch.no_grad():
        outputs = model(input_ids, token_type_ids=None,
        attention_mask=input_mask,)
        # For eval mode, the first result of outputs is logits
        logits = outputs[0] 
    
    # Get NER predict result
    logits = torch.argmax(F.log_softmax(logits,dim=2),dim=2)
    logits = logits.detach().cpu().numpy()
    
    
    # Get NER true result
    label_ids = label_ids.to('cpu').numpy()
    
    
    # Only predict the real word, mark=0, will not calculate
    input_mask = input_mask.to('cpu').numpy()
    
    # Compare the valuable predict result
    for i,mask in enumerate(input_mask):
        # Real one
        temp_1 = []
        # Predict one
        temp_2 = []
        
        for j, m in enumerate(mask):
            # Mark=0, meaning its a pad word, dont compare
            if m:
                if tag2name[label_ids[i][j]] != "X" and tag2name[label_ids[i][j]] != "[CLS]" and tag2name[label_ids[i][j]] != "[SEP]" : # Exclude the X label
                    temp_1.append(tag2name[label_ids[i][j]])
                    temp_2.append(tag2name[logits[i][j]])
            else:
                break
        
            
        y_true.append(temp_1)
        y_pred.append(temp_2)

        

print("f1 socre: %f"%(f1_score(y_true, y_pred)))
print("Accuracy score: %f"%(accuracy_score(y_true, y_pred)))

# Get acc , recall, F1 result report
report = classification_report(y_true, y_pred,digits=4)

# Save the report into file
# output_eval_file = os.path.join('/content/drive/MyDrive/BERT/NER Corpus/bert-base-multilingual-cased', "eval_results.txt")
output_eval_file = os.path.join(bert_out_address, "eval_results.txt")
with open(output_eval_file, "w") as writer:
    print("***** Eval results *****")
    print("\n%s"%(report))
    print("f1 socre: %f"%(f1_score(y_true, y_pred)))
    print("Accuracy score: %f"%(accuracy_score(y_true, y_pred)))
    
    writer.write("f1 socre:\n")
    writer.write(str(f1_score(y_true, y_pred)))
    writer.write("\n\nAccuracy score:\n")
    writer.write(str(accuracy_score(y_true, y_pred)))
    writer.write("\n\n")  
    writer.write(report)

***** Running evaluation *****
  Num examples =2843
  Batch size = 32


/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: o seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: X seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: [SEP] seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))


f1 socre: 0.469522
Accuracy score: 0.855230


/usr/local/lib/python3.7/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


***** Eval results *****

               precision    recall  f1-score   support

     ARTIFACT     0.3333    0.2281    0.2708        57
        COUNT     0.7709    0.8412    0.8045       340
         DATE     0.0000    0.0000    0.0000         4
          DAY     0.0000    0.0000    0.0000         3
      DISEASE     0.2857    0.1739    0.2162        23
     DISTANCE     0.2000    0.2857    0.2353        14
ENTERTAINMENT     0.5455    0.4660    0.5026       103
   FACILITIES     0.0000    0.0000    0.0000        12
    LIVTHINGS     0.3571    0.2632    0.3030        19
     LOCATION     0.3909    0.4624    0.4236       372
   LOCOMOTIVE     1.0000    0.2857    0.4444         7
    MATERIALS     0.3000    0.2222    0.2553        27
        MONTH     0.4545    0.3333    0.3846        15
 ORGANIZATION     0.1875    0.1034    0.1333        29
       PERIOD     0.3235    0.4074    0.3607        27
       PERSON     0.5000    0.3833    0.4339       347
       PLANTS     0.0000    0.0000    

In [ ]:
save_model_address = 'ashiNLP/bert_hindberto_model/hindi'

In [ ]:
save_model = AutoModelForMaskedLM.from_pretrained(save_model_address,num_labels=len(tag2idx))

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(save_model_address,do_lower_case=False)

In [ ]:
# predict
test_query = "मैं 20 साल से दिल्ली में रहता हूं"

In [ ]:
tokenized_texts = []

temp_token = []
temp_token.append('[CLS]')
token_list = tokenizer.tokenize(test_query)

token_list

['à¤',
 '®',
 'à¥',
 'Ī',
 'à¤',
 'Ĥ',
 'Ġ20',
 'Ġà¤',
 '¸',
 'à¤¾',
 'à¤',
 '²',
 'Ġà¤',
 '¸',
 'à¥ĩ',
 'Ġà¤',
 '¦',
 'à¤',
 '¿',
 'à¤',
 '²',
 'à¥į',
 'à¤',
 '²',
 'à¥Ģ',
 'Ġà¤',
 '®',
 'à¥ĩ',
 'à¤',
 'Ĥ',
 'Ġà¤',
 '°',
 'à¤',
 '¹',
 'à¤',
 '¤',
 'à¤¾',
 'Ġà¤',
 '¹',
 'à¥',
 'Ĥ',
 'à¤',
 'Ĥ']

In [ ]:
for m,token in enumerate(token_list):
    temp_token.append(token)

In [ ]:
if len(temp_token) > max_len-1:
    temp_token= temp_token[:max_len-1]

In [ ]:
temp_token.append('[SEP]')
temp_token

['[CLS]',
 'à¤',
 '®',
 'à¥',
 'Ī',
 'à¤',
 'Ĥ',
 'Ġ20',
 'Ġà¤',
 '¸',
 'à¤¾',
 'à¤',
 '²',
 'Ġà¤',
 '¸',
 'à¥ĩ',
 'Ġà¤',
 '¦',
 'à¤',
 '¿',
 'à¤',
 '²',
 'à¥į',
 'à¤',
 '²',
 'à¥Ģ',
 'Ġà¤',
 '®',
 'à¥ĩ',
 'à¤',
 'Ĥ',
 'Ġà¤',
 '°',
 'à¤',
 '¹',
 'à¤',
 '¤',
 'à¤¾',
 'Ġà¤',
 '¹',
 'à¥',
 'Ĥ',
 'à¤',
 'Ĥ',
 '[SEP]']

In [ ]:
tokenized_texts.append(temp_token)

In [ ]:
input_ids = pad_sequences([tokenizer.convert_tokens_to_ids(txt) for txt in tokenized_texts],
                          maxlen=max_len, dtype="long", truncating="post", padding="post")
print(input_ids[0])


[    3  6843   111  9916   235  6843   229   734 14614   121 21507  6843
   115 14614   121 29892 14614   104  6843   128  6843   115 28023  6843
   115 30287 14614   111 29892  6843   229 14614   113  6843   122  6843
   102 21507 14614   122  9916   229  6843   229     3]


In [ ]:
# For fine tune of predict, with token mask is 1,pad token is 0
attention_masks = [[int(i>0) for i in ii] for ii in input_ids]
attention_masks[0];

In [ ]:

segment_ids = [[0] * len(input_id) for input_id in input_ids]
segment_ids[0];


In [ ]:
input_ids = torch.tensor(input_ids)
attention_masks = torch.tensor(attention_masks)
segment_ids = torch.tensor(segment_ids)

In [ ]:
model.eval();

In [ ]:
with torch.no_grad():
        outputs = save_model(input_ids, token_type_ids=None,
        attention_mask=None,)
        # For eval mode, the first result of outputs is logits
        logits = outputs[0] 

In [ ]:
predict_results = logits.detach().cpu().numpy()
predict_results.shape


(1, 45, 30522)

In [ ]:
from scipy.special import softmax
result_arrays_soft = softmax(predict_results[0])
result_arrays_soft[0]


array([3.4210159e-13, 1.8820103e-13, 5.8559746e-10, ..., 1.5407079e-14,
       1.5783559e-13, 4.6627822e-14], dtype=float32)

In [ ]:
result_array = result_arrays_soft
len(result_array),len(result_array[0])

(45, 30522)

In [ ]:
# Get each token final predict tag index result
result_list = np.argmax(result_array,axis=-1)
result_list

array([43, 15, 41, 41, 41, 41, 41, 22, 13, 41, 41, 41, 41, 42, 41, 41,  0,
       41, 41, 41, 41, 41, 41, 41, 41, 41, 42, 41, 41, 41, 41, 42, 41, 41,
       41, 41, 41, 41, 42, 41, 41, 41, 41, 41, 44])

In [ ]:
for i, mark in enumerate(attention_masks[0]):
    if mark>0:
        print("Token:%s"%(temp_token[i]))
#         print("Tag:%s"%(result_list[i]))
        print("Predict_Tag:%s"%(tag2name[result_list[i]]))
        #print("Posibility:%f"%(result_array[i][result_list[i]]))
        print()

Token:[CLS]
Predict_Tag:[CLS]

Token:à¤
Predict_Tag:B-PERSON

Token:®
Predict_Tag:X

Token:à¥
Predict_Tag:X

Token:Ī
Predict_Tag:X

Token:à¤
Predict_Tag:X

Token:Ĥ
Predict_Tag:X

Token:Ġ20
Predict_Tag:B-COUNT

Token:Ġà¤
Predict_Tag:I-PERIOD

Token:¸
Predict_Tag:X

Token:à¤¾
Predict_Tag:X

Token:à¤
Predict_Tag:X

Token:²
Predict_Tag:X

Token:Ġà¤
Predict_Tag:o

Token:¸
Predict_Tag:X

Token:à¥ĩ
Predict_Tag:X

Token:Ġà¤
Predict_Tag:B-LOCATION

Token:¦
Predict_Tag:X

Token:à¤
Predict_Tag:X

Token:¿
Predict_Tag:X

Token:à¤
Predict_Tag:X

Token:²
Predict_Tag:X

Token:à¥į
Predict_Tag:X

Token:à¤
Predict_Tag:X

Token:²
Predict_Tag:X

Token:à¥Ģ
Predict_Tag:X

Token:Ġà¤
Predict_Tag:o

Token:®
Predict_Tag:X

Token:à¥ĩ
Predict_Tag:X

Token:à¤
Predict_Tag:X

Token:Ĥ
Predict_Tag:X

Token:Ġà¤
Predict_Tag:o

Token:°
Predict_Tag:X

Token:à¤
Predict_Tag:X

Token:¹
Predict_Tag:X

Token:à¤
Predict_Tag:X

Token:¤
Predict_Tag:X

Token:à¤¾
Predict_Tag:X

Token:Ġà¤
Predict_Tag:o

Token:¹
Predict_Tag:X

Token:à